In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# ------------------- Parameters (ns units) -------------------
kT = 0.025  # eV
density = 1.0  # sites/nm^3
L = 10.0  # nm
nu0 = 1e3  # ns^-1
lambda_eV = 16*kT
rcut = 3.0
max_hops = 1000
n_traj = 50

# Sigma and alpha grids
sigma_kT_factors = np.arange(1, 5.01, 0.5)  # 1,1.5,...,5 kT
alpha_list = np.arange(0.1, 0.61, 0.05)     # nm

# ------------------- Helpers -------------------
def minimum_image(vec, L):
    return vec - L*np.round(vec/L)

def build_sites(N_sites,L):
    return np.random.rand(N_sites,3)*L

def build_neighbor_list(positions, rcut, L):
    try:
        from scipy.spatial import cKDTree as KDTree
        tree = KDTree(positions, boxsize=L)
        neighbors = tree.query_ball_tree(tree, rcut)
        neighbors = [[j for j in nbrs if j!=i] for i,nbrs in enumerate(neighbors)]
        return neighbors
    except:
        N = len(positions)
        nbrs_all=[]
        for i in range(N):
            dvec = positions - positions[i]
            dvec = minimum_image(dvec,L)
            dists = np.sqrt((dvec**2).sum(axis=1))
            nbrs = np.where((dists<=rcut) & (dists>1e-12))[0].tolist()
            nbrs_all.append(nbrs)
        return nbrs_all

# ------------------- Rates -------------------
def rate_MA(Ei,Ej,r,alpha):
    return nu0*np.exp(-2*r/alpha)*(np.exp(-(Ej-Ei)/kT) if Ej>Ei else 1.0)

def rate_Marcus(Ei,Ej,r,alpha):
    return nu0*np.exp(-2*r/alpha)*np.exp(-((Ej-Ei+lambda_eV)**2)/(4*lambda_eV*kT))

# ------------------- Trajectory -------------------
def run_trajectory(positions, energies, neighbors, alpha, rate_type="MA", max_hops=1000):
    N = len(positions)
    cur = np.random.randint(N)
    pos0 = positions[cur].copy()
    t_total = 0.0  # ns
    for step in range(max_hops):
        nbrs = neighbors[cur]
        if len(nbrs)==0: break
        dvec = positions[nbrs]-positions[cur]
        dvec = minimum_image(dvec,L)
        rdist = np.sqrt((dvec**2).sum(axis=1))
        if rate_type=="MA":
            rates = np.array([rate_MA(energies[cur], energies[j], r, alpha) for j,r in zip(nbrs,rdist)])
        else:
            rates = np.array([rate_Marcus(energies[cur], energies[j], r, alpha) for j,r in zip(nbrs,rdist)])
        Rtot = rates.sum()
        if Rtot<=0: break
        dt = -np.log(np.random.rand())/Rtot  # ns
        t_total += dt
        probs = rates/Rtot
        idx = np.searchsorted(np.cumsum(probs), np.random.rand(), side='right')
        if idx>=len(nbrs): idx=len(nbrs)-1
        cur = nbrs[idx]
    d = positions[cur]-pos0
    d = minimum_image(d,L)
    rsq = (d**2).sum()
    return rsq, t_total

# ------------------- Study -------------------
def run_study():
    N_sites = int(density*L**3)
    positions = build_sites(N_sites,L)
    neighbors = build_neighbor_list(positions, rcut,L)

    D_data = np.zeros((len(sigma_kT_factors), len(alpha_list), 2))  # 0=MA,1=Marcus
    total_runs = len(sigma_kT_factors) * len(alpha_list) * 2
    run_count = 0
    start_time = time.time()

    for i_sigma, sigma_factor in enumerate(sigma_kT_factors):
        sigma = sigma_factor*kT
        energies = np.random.normal(0,sigma,N_sites)
        for i_alpha, alpha in enumerate(alpha_list):
            for j_rate, rate_type in enumerate(["MA","Marcus"]):
                run_count += 1
                elapsed = time.time() - start_time
                remaining = elapsed / run_count * (total_runs - run_count)
                print(f"[{run_count}/{total_runs}] sigma={sigma_factor} kT, alpha={alpha:.2f} nm, rate={rate_type} | "
                      f"Elapsed: {elapsed:.1f}s, Remaining: {remaining:.1f}s")

                rsq_list=[]
                t_list=[]
                for _ in range(n_traj):
                    rsq, t_total = run_trajectory(positions, energies, neighbors, alpha, rate_type, max_hops)
                    rsq_list.append(rsq)
                    t_list.append(t_total)
                avg_rsq = np.mean(rsq_list)
                avg_t_ns = np.mean(t_list)
                D = avg_rsq/(6*avg_t_ns)
                D_data[i_sigma, i_alpha, j_rate] = D

    return D_data

# ------------------- Run study -------------------
D_data = run_study()

# ------------------- Save as numpy file -------------------
np.savez("kmc_diffusion_data.npz", D=D_data, alpha=alpha_list, sigma=sigma_kT_factors)

# ------------------- Plot -------------------
plt.figure(figsize=(7,5))
for i_sigma, sigma_factor in enumerate(sigma_kT_factors):
    for j_rate, rate_type in enumerate(["MA","Marcus"]):
        plt.plot(alpha_list, D_data[i_sigma,:,j_rate], marker='o', label=f"{rate_type}, σ={sigma_factor}kT")
plt.xlabel("Localization length α (nm)")
plt.ylabel("Diffusion coefficient D (nm²/ns)")
plt.yscale('log')
plt.title("Diffusion coefficient vs localization length")
plt.legend(fontsize=8)
plt.tight_layout()
plt.show()


[1/198] sigma=1.0 kT, alpha=0.10 nm, rate=MA | Elapsed: 0.0s, Remaining: 0.0s
[2/198] sigma=1.0 kT, alpha=0.10 nm, rate=Marcus | Elapsed: 26.2s, Remaining: 2570.3s
[3/198] sigma=1.0 kT, alpha=0.15 nm, rate=MA | Elapsed: 52.0s, Remaining: 3378.0s


KeyboardInterrupt: 